In [13]:
# smile 뜨게 만드는 작업 했어욯ㅎㅎ
import cv2
import sys
import numpy as np

faceCascade = cv2.CascadeClassifier('D:/javaForever/util/opencv/sources/data/haarcascades/haarcascade_frontalface_alt.xml')
eyeCascade = cv2.CascadeClassifier('D:/javaForever/util/opencv/sources/data/haarcascades/haarcascade_eye.xml')
mouthCascade = cv2.CascadeClassifier('D:/javaForever/util/opencv/sources/data/haarcascades/haarcascade_mcs_mouth.xml')

video_capture = cv2.VideoCapture('facevideo.mp4')

def findMouth(frame, eyes, faces):
    mouths = mouthCascade.detectMultiScale(
            frame,
            scaleFactor=1.2,
            minNeighbors=8,
            minSize=(30, 10),
            flags = cv2.CASCADE_SCALE_IMAGE)

    if len(mouths) <= 0:
        return 0, 0, 0, 0

    eyeBottom = 0

    for eye in eyes:
        eyeX, eyeY, eyeW, eyeH = eye
        if (eyeY + eyeH) > eyeBottom:
            eyeBottom = eyeY + eyeH

    faceX, faceY, faceW, faceH = faces[0]
    faceBottom = faceY + faceH

    for mouth in mouths:
        mouthX, mouthY, mouthW, mouthH = mouth
        if mouthY > (eyeBottom + 50) and (mouthY + mouthH) < faceBottom:
            return mouth

    return 0, 0, 0, 0

if __name__ == '__main__':
    while True:
        # Capture frame-by-frame
        ret, frame = video_capture.read()

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray2 = cv2.equalizeHist(gray)

        faces = faceCascade.detectMultiScale(
            gray,
            scaleFactor=1.2,
            minNeighbors=6,
            minSize=(50, 50),
            flags = cv2.CASCADE_SCALE_IMAGE)

        eyes = eyeCascade.detectMultiScale(
            gray,
            scaleFactor=1.2,
            minNeighbors=6,
            minSize=(50, 50),
            flags = cv2.CASCADE_SCALE_IMAGE)


        # Draw a rectangle around the faces
        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            
        # Draw a rectangle around the eyes
        if len(eyes) >= 2:
            eyes = eyes[:2]
        for (x, y, w, h) in eyes:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)
        
        # Draw a rectangle around the mouth
        if len(faces) > 0 and len(eyes) > 0:
            x, y, w, h = findMouth(gray2, eyes, faces)
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
            cv2.putText(frame,'Smile',(x,y-7), 3, 1.2, (0, 255, 0), 2, cv2.LINE_AA)
            
        # Display the resulting frame
        cv2.imshow('Video', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # When everything is done, release the capture
video_capture.release()
cv2.destroyAllWindows()

In [16]:
import cv2
import numpy as np
import math
cap = cv2.VideoCapture('CHARLIE PUTH - How Long  Kyle Hanagami Choreography')

while(cap.isOpened()):
    ret, img = cap.read()
    cv2.rectangle(img,(300,300),(100,100),(0,255,0),0)
    crop_img = img[100:300, 100:300]
    grey = cv2.cvtColor(crop_img, cv2.COLOR_BGR2GRAY)
    value = (35, 35)
    blurred = cv2.GaussianBlur(grey, value, 0)
    _, thresh1 = cv2.threshold(blurred, 127, 255,
                               cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    cv2.imshow('Thresholded', thresh1)
    contours, hierarchy = cv2.findContours(thresh1.copy(),cv2.RETR_TREE, \
            cv2.CHAIN_APPROX_NONE)
    max_area = -1
    for i in range(len(contours)):
        cnt=contours[i]
        area = cv2.contourArea(cnt)
        if(area>max_area):
            max_area=area
            ci=i
    cnt=contours[ci]
    x,y,w,h = cv2.boundingRect(cnt)
    cv2.rectangle(crop_img,(x,y),(x+w,y+h),(0,0,255),0)
    hull = cv2.convexHull(cnt)
    drawing = np.zeros(crop_img.shape,np.uint8)
    cv2.drawContours(drawing,[cnt],0,(0,255,0),0)
    cv2.drawContours(drawing,[hull],0,(0,0,255),0)
    hull = cv2.convexHull(cnt,returnPoints = False)
    defects = cv2.convexityDefects(cnt,hull)
    count_defects = 0
    cv2.drawContours(thresh1, contours, -1, (0,255,0), 3)
    for i in range(defects.shape[0]):
        s,e,f,d = defects[i,0]
        start = tuple(cnt[s][0])
        end = tuple(cnt[e][0])
        far = tuple(cnt[f][0])
        a = math.sqrt((end[0] - start[0])**2 + (end[1] - start[1])**2)
        b = math.sqrt((far[0] - start[0])**2 + (far[1] - start[1])**2)
        c = math.sqrt((end[0] - far[0])**2 + (end[1] - far[1])**2)
        angle = math.acos((b**2 + c**2 - a**2)/(2*b*c)) * 57
        if angle <= 90:
            count_defects += 1
            cv2.circle(crop_img,far,1,[0,0,255],-1)
        #dist = cv2.pointPolygonTest(cnt,far,True)
        cv2.line(crop_img,start,end,[0,255,0],2)
        #cv2.circle(crop_img,far,5,[0,0,255],-1)
    if count_defects == 1:
        cv2.putText(img,"I'M VIKRANT SHARMA", (50,50), cv2.FONT_HERSHEY_SIMPLEX, 2, 2)
    elif count_defects == 2:
        str = "THIS IS A BASIC HAND GESTURE RECOGNISER!!"
        cv2.putText(img, str, (5,50), cv2.FONT_HERSHEY_SIMPLEX, 1, 2)
    elif count_defects == 3:
        cv2.putText(img,"This is FOUR (:", (50,50), cv2.FONT_HERSHEY_SIMPLEX, 2, 2)
    elif count_defects == 4:
        cv2.putText(img,"HARE KRSNA", (50,50), cv2.FONT_HERSHEY_SIMPLEX, 2, 2)
    else:
        cv2.putText(img,"GOOD AFTERNOON TEACHERS", (50,50),\
                    cv2.FONT_HERSHEY_SIMPLEX, 2, 2)
    #cv2.imshow('drawing', drawing)
    #cv2.imshow('end', crop_img)
    cv2.imshow('Gesture', img)
    all_img = np.hstack((drawing, crop_img))
    cv2.imshow('Contours', all_img)
    k = cv2.waitKey(10)
    if k == 27:
        break

TypeError: 'NoneType' object is not subscriptable